<a href="https://colab.research.google.com/github/Duke-BC-DL-for-Energy-Infrastructure/Colab-Notebooks-for-Training-Models/blob/main/Few%20Shot%20Learning/Train_NE_Val_NE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!git clone https://github.com/Duke-BC-DL-for-Energy-Infrastructure/yolov3.git  # clone our fork of the ultralytics repo

In [ ]:
!mkdir -p data/

# get our data and unzip it
!wget -nc -O data/images.zip https://duke.box.com/shared/static/rttn4ujt4h9sbuba4poibdvtlqr7fhvz.zip
!wget -nc -O data/labels.zip https://duke.box.com/shared/static/psea4em0oh1ng99neowslj9hfz79sckn.zip

!unzip -n -q data/images.zip -d data
!unzip -n -q data/labels.zip -d data

# get the synthetic images and labels and unzip those into the same directory
!wget -nc -O data/synthetic_images.zip https://duke.box.com/shared/static/o82ap1y9h1tgm6dwy32wzqnz0y4fycee.zip
!wget -nc -O data/synthetic_labels.zip https://duke.box.com/shared/static/1dbmv9qmof4gbxkm0pczgnm5uz90mkym.zip

!unzip -n -q data/synthetic_images.zip -d data
!unzip -n -q data/synthetic_labels.zip -d data

In [ ]:
# get modified cfg file that works for 1 class
!rm -rf yolov3/cfg 

!wget -nc -O yolov3/cfg.zip https://duke.box.com/shared/static/pq52dolw5lo5ds8fd7g9v72ekzlngekz.zip
!unzip -n -q yolov3/cfg.zip -d yolov3

In [ ]:
# GET DATA
# delete the original data folder so we can unzip our own data folder with the wind turbine data
!rm -rf yolov3/data

# load in the baseline data files from the box
!wget -nc -O yolov3/baseline_10.zip https://duke.box.com/shared/static/1g33jvwi6t86zx81ihut83oe09phknke.zip
!unzip -n -q yolov3/baseline_10.zip -d yolov3

!wget -nc -O yolov3/baseline_20.zip https://duke.box.com/shared/static/5mqyzea5w6vom9rsguj26l0tra6q3rwn.zip
!unzip -n -q yolov3/baseline_20.zip -d yolov3

# get the synthetic data files from the box
!wget -nc -O yolov3/adding_synthetic_0.zip https://duke.box.com/shared/static/o9l3u576qt32z9z5ysbalx7jsv9ajqvv.zip
!unzip -n -q yolov3/adding_synthetic_0.zip -d yolov3

!wget -nc -O yolov3/adding_synthetic_10.zip https://duke.box.com/shared/static/3pcvitn0b2rcwumfch0hglyywdtfbcm3.zip
!unzip -n -q yolov3/adding_synthetic_10.zip -d yolov3

!wget -nc -O yolov3/adding_synthetic_20.zip https://duke.box.com/shared/static/kckidwulzv63yuer6piyibftxswmqdx9.zip
!unzip -n -q yolov3/adding_synthetic_20.zip -d yolov3

In [ ]:
%pwd

In [ ]:
%cd yolov3

In [ ]:
# Training on 10 NE real images, validating on 200 NE images
!python3 train.py --cfg cfg/yolov3-spp.cfg --data baseline_10/baseline.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data baseline_10/baseline.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [ ]:
# Training on 20 NE real images, validating on 200 NE images
!python3 train.py --cfg cfg/yolov3-spp.cfg --data baseline_20/baseline.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data baseline_20/baseline.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [ ]:
# Training on 0 real NE and 100 synthetic NE, validating on 200 real NE images
!python3 train.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_0/adding_synthetic.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_0/adding_synthetic.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [ ]:
# Training on 10 real NE and 100 synthetic NE, validating on 200 real NE images
!python3 train.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_10/adding_synthetic.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_10/adding_synthetic.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [ ]:
# Training on 20 real NE and 100 synthetic NE, validating on 200 real NE images
!python3 train.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_20/adding_synthetic.data --img-size 608 --epochs 300 --batch-size 10 --device 0

In [ ]:
!python3 test.py --cfg cfg/yolov3-spp.cfg --data adding_synthetic_20/adding_synthetic.data --img-size 608 --weights weights/last.pt --conf-thres 0.0001

In [ ]:
# Create folder for validation dataset, so that we can run detect.py on it

VAL_PATH = 'baseline_10/val_img_paths.txt' # Provide path to .txt file for validation set

from shutil import copyfile
%mkdir ../data/val

with open(VAL_PATH, 'r') as f:
  val_imgs =  [path.replace('\n', '') for path in f.readlines()]

for img in val_imgs:
  # copy image to val folder
  copyfile(img, img.replace('images', 'val'))

In [ ]:
# DETECT using last.pt, the last weights that were trained
!python3 detect.py --names baseline_10/wnd.names --weights weights/last.pt --source ../data/val --img-size 608